In [ ]:
import logging
from pathlib import Path

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import rioxarray as rxr
import xarray as xr
import pandas as pd
from affine import Affine
from geopandas.geodataframe import GeoDataFrame
from rasterio.features import rasterize
from sklearn.ensemble import RandomForestClassifier

from training_raster_clipper.core.logging import log_info
from training_raster_clipper.core.models import TrainingConfiguration, TrainingFunctions
from training_raster_clipper.core.visualization import (
    plot_array,
    plot_geodataframe,
    plot_rgb_data_array,
)
from training_raster_clipper.custom_types import (
    BandNameType,
    ClassificationResult,
    ClassifiedSamples,
    FeatureClassNameToId,
    PolygonMask,
    ResolutionType,
)

import logging


In [ ]:
# See https://stackoverflow.com/questions/18786912/get-output-from-the-logging-module-in-ipython-notebook
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.info("test")


In [ ]:
# user = "eschalk"
# user = "eschalk_debug"
user = "emunaibari"

if user == "emunaibari":
    raster_input_path = Path(
        "D:/Trainings/HW/Data/"
    )
    polygons_input_path = Path(
        "D:/Trainings/HW/Ploygon/Polygons.geojson"
    )
    output_path = Path('D:/Trainings/HW/generated/')
elif user == "eschalk":
    raster_input_path = Path(
        r"..\resources\solution\example_sentinel_files\SENTINEL.SAFE\GRANULE\L2A_T31TCJ_A038658_20221116T105603\IMG_DATA\R60m"    )
    polygons_input_path = Path(
        r"D:\Profils\eschalk\dev\playground\python\training\training-raster-clipper\resources\your_work\Polygons_emunaibari.geojson"
    )
    output_path = Path('../generated/emunaibari')
elif user == "eschalk_debug":
    raster_input_path = Path(
        r"..\resources\solution\example_sentinel_files\SENTINEL.SAFE\GRANULE\L2A_T31TCJ_A038658_20221116T105603\IMG_DATA\R60m"    )
    polygons_input_path=(
        Path(".").resolve().parent / Path("resources/solution/polygons.geojson")
    )
    output_path = Path('../generated/emunaibari')
    
else: 
    raster_input_path = None
    polygons_input_path = None
    output_path = None

assert raster_input_path.exists() and raster_input_path.is_dir()
assert polygons_input_path.exists() and polygons_input_path.is_file()

if not (output_path.exists() and output_path.is_dir()):
    output_path.mkdir()

config = TrainingConfiguration(
    verbose=True,
    show_plots=True,
    resolution=60,
    band_names=("B04", "B03", "B02", "B8A"),
    raster_input_path=raster_input_path,
    polygons_input_path=polygons_input_path,
    csv_output_path = (output_path.resolve() / Path("classified_points.csv")),
    raster_output_path=(output_path.resolve() / Path("sklearn_raster.tiff")),
    implementation_name="eschalk",
)
config


In [ ]:
verbose = config.verbose
show_plots = config.show_plots

resolution = config.resolution
band_names = config.band_names

raster_input_path = config.raster_input_path
polygons_input_path = config.polygons_input_path
csv_output_path = config.csv_output_path
raster_output_path = config.raster_output_path


### (1) Load a GeoJSON file with `geopandas`


In [ ]:
def load_feature_polygons(input_path: Path) -> GeoDataFrame:
    # Loading the Polygons
    poly_file = gpd.read_file(input_path)
    # Converting the coordinate system from EPSG 4326 format, to the Sentinel-2 raster: 32631
    poly_file = poly_file.to_crs(32631)

    return poly_file


In [ ]:
polygons = load_feature_polygons(polygons_input_path)
if verbose:
    log_info(polygons, "polygons")
if show_plots:
    plot_geodataframe(polygons, f"{load_feature_polygons.__name__}")


### (2) Load a Sentinel-2 raster with `rioxarray`


In [ ]:
def load_sentinel_data(
    sentinel_product_location: Path,
    resolution: ResolutionType,
    band_names: tuple[BandNameType, ...],
) -> xr.DataArray:
    """Loads sentinel product

    Example input path: `S2A_MSIL2A_20221116T105321_N0400_R051_T31TCJ_20221116T170958.SAFE`

    Args:
        sentinel_product_location (Path): Location of the .SAFE folder containing a Sentinel-2 product.

    Returns:
        xr.DataArray: A DataArray containing the 3 RGB bands from the visible spectrum
    """

    # Reading each band into an xarray DataArray and store in a dictionary, using dictionary comprehension
    band_data = {
        band: rxr.open_rasterio(
            next(sentinel_product_location.glob(f"*{band}*")), masked=True
        ).squeeze()
        for band in band_names
    }

    # The .squeeze() is used to remove dimensions of size 1 from an xarray DataArray or Dataset.
    # When you read a single-band raster file using rioxarray or rasterio, it usually comes in with dimensions like (1, height, width).
    # The masked=True argument is used when opening the raster file with rioxarray to automatically convert any NoData values in the raster to NaN (Not a Number) in the resulting xarray DataArray.
    # rioxarray and rasterio can automatically detect NoData values based on the raster file's metadata.

    # Concatenating the individual bands into a single xarray DataArray along a new 'band' dimension
    all_bands = xr.concat(list(band_data.values()), dim="band")
    all_bands["band"] = list(
        band_data.keys()
    )  # Assigning band names to the 'band' coordinate

    # Converting the data type to np.float32
    all_bands = all_bands.astype(np.float32)

    # Pre-processing on rasters
    offset = -1000.0
    quantification = 10000.0
    # Normalization
    all_bands = (all_bands + offset) / quantification

    return all_bands

In [ ]:
rasters = load_sentinel_data(raster_input_path, resolution, band_names)
if verbose:
    log_info(rasters, "rasters")
if show_plots:
    plot_rgb_data_array(rasters, f"{load_sentinel_data.__name__}")


### (3) Rasterize the polygons


In [ ]:
def rasterize_geojson(
    data_array: xr.DataArray,
    training_classes: GeoDataFrame,
) -> tuple[PolygonMask, FeatureClassNameToId]:
    """Burns a set of vectorial polygons to a raster.

    See https://gis.stackexchange.com/questions/316626/rasterio-features-rasterize

    Args:
        data_array (xr.DataArray): The Sentinel raster, from which data is taken, such as the transform or the shape.
        training_classes (GeoDataFrame): The input set of classified multipolygons to burn

    Returns:
        xr.DataArray: A mask raster generated from the polygons, representing the same geographical region as the source dataarray param
                      0 where no polygon were found, and integers representing classes in order of occurence in the GeoDataFrame
    """

    # extracting the shape from the raster
    raster_shape = data_array.isel(band=0).shape

    # the transform
    transform = rasters.spatial_ref.GeoTransform
    # the output shows a string displaying a GDAL-style geotransform parameters (six parameters):
    # -a: top left x coordinate (longitude)
    # -b: width of a pixel in x direction
    # -c: rotation (typically 0)
    # -d: top left y coordinate (latitude)
    # -e: rotation (typically 0)
    # -f: height of a pixel (in y direction, usually negative)

    # Getting the different paramaters and convert them to floats instead of strings
    transform = [float(a) for a in transform.split()]

    # Converting the transform to the required format (Affine) by feature.rasterize
    transform = Affine.from_gdal(*transform)

    # Getting the Polygons geometry
    # geom = [
    #     (g, v)
    #     for g, v in zip(training_classes.geometry, training_classes.id.values + 1)
    # ]
    geom = [(g, v) for g, v in zip(training_classes.geometry, training_classes.index + 1)]

    # Getting the classes
    # unique_classes = dict(
    #     zip(training_classes["class"].values, training_classes.id.values + 1)
    # )
    unique_classes = dict(zip(training_classes["class"].values, training_classes.index + 1))
    cls = {cl: val for cl, val in unique_classes.items()}

    # Mask building: resizing and geomtery encoding. Ensuring the mask will have the correct designated type (PolygonMask)
    mask: PolygonMask = rasterize(
        geom,
        out_shape=raster_shape,
        transform=transform,
        all_touched=False,
        dtype=np.uint8,
    )

    return [mask, cls]

In [ ]:
burnt_polygons, mapping = rasterize_geojson(rasters, polygons)
if verbose:
    log_info(burnt_polygons, "burnt_polygons")
    log_info(mapping, "mapping")
if show_plots:
    plot_array(burnt_polygons, f"{rasterize_geojson.__name__}")


### (4) Intersect the Sentinel-2 raster with polygons


In [ ]:
def produce_clips(
    data_array: xr.DataArray, burnt_polygons: PolygonMask, mapping: FeatureClassNameToId
) -> ClassifiedSamples:
    """Extract RGB values covered by classified polygons

    Args:
        data_array (xr.DataArray): RGB raster
        burnt_polygons (PolygonMask): Rasterized classified multipolygons

    Returns:
        _type_: A list of the RGB values contained in the data_array and their corresponding classes
    """
    # To host a new xarray.Dataset containing the reflectance and the features ID
    ref_feature = None
    # Getting the features IDs from the mapping dictionary
    for _, id_feature in mapping.items():
        # Flattening the rasters reflectance and assign it to new dimension variable z
        # ref = data_array.stack(z=("x", "y"))
        ref = data_array.stack(z=("y", "x"))
        # Flattening the mask
        poly = burnt_polygons.flatten()
        # Using the mask to select the reflectance pixels under the current feature in the loop (water, forest, or farmland)
        ref = ref.isel(z=poly == id_feature)
        # Removing un-needed variables
        ref = ref.drop_vars(("y", "x", "spatial_ref"))

        # Putting the id feature into an xr.Dataset similar the selected reflectance
        feature_arr = xr.ones_like(ref.z) * id_feature

        # Combining the reflectance data, if the xr.Dataset exist or generating a one if not
        if ref_feature:
            ref_feature = xr.concat(
                [
                    ref_feature,
                    xr.Dataset({"reflectance": ref, "feature_id": feature_arr}),
                ],
                dim="z",
            )
        else:
            ref_feature = xr.Dataset({"reflectance": ref, "feature_id": feature_arr})

    return ref_feature

In [ ]:
classified_rgb_rows = produce_clips(rasters, burnt_polygons, mapping)
if verbose:
    log_info(classified_rgb_rows, "classified_rgb_rows")


In [ ]:
classified_rgb_rows.reflectance.sel(band="B04").plot()
classified_rgb_rows.reflectance.sel(band="B03").plot()
classified_rgb_rows.reflectance.sel(band="B02").plot()
classified_rgb_rows.reflectance.sel(band="B8A").plot()


### (5) Persist the intersection to a CSV


In [ ]:
def persist_to_csv(
    classified_rgb_rows: ClassifiedSamples,
    csv_output_path: Path,
) -> None:
    # Converting the reflectance DataArray into pandas.DataFrame after transposing the DataArray so we have four columns corresponding to the four bands
    pd_df = classified_rgb_rows.reflectance.T.to_pandas()
    # Adding the features IDs to the DataFrame
    pd_df["feature_id"] = classified_rgb_rows.feature_id.data

    # Saveing the DataFrame as CSV file with separator ';'
    pd_df.to_csv(csv_output_path, sep=";", index=False)


In [ ]:
persist_to_csv(classified_rgb_rows, csv_output_path)
log_info(f"Written CSV output {csv_output_path}")


### (6) Train a machine learning model


In [ ]:
def classify_sentinel_data(
    rasters: xr.DataArray, classified_rgb_rows: ClassifiedSamples
) -> ClassificationResult:
    # Getting the features reflectance and the labels, the current shape of the reflectance array is 4xn (row: bands, col: ref), so we transpose it to get nx4 (row: ref, col: bands)
    feature_ref = classified_rgb_rows.reflectance.T
    feature_label = classified_rgb_rows.feature_id

    # Initiating the classifier, n_job is to speed the process through parallelization
    classifier = RandomForestClassifier(n_jobs=4)  # n_estimators=100, random_state=42)

    # Training the model in the features reflectance and the labels
    classifier.fit(feature_ref, feature_label)

    # Preparing the rasters for prediction by flattening the reflectance data assign it to new dimension variable z and transpose it to match the train data
    flt_data = rasters.stack(z=("y", "x")).T
    # flt_data = rasters.stack(z=("x", "y")).T

    # Predicting the classes of the rasters
    prd_classes = classifier.predict(flt_data)

    # Reshaping the prediction result to match the original shape and put it into [through .copy(data=)] xr.DataArry with the same metadata and attribute as the rasters
    xr_holder = rasters.isel(band=0, drop=True)
    xr_holder = xr_holder.transpose("x", "y") # debug 
    classifier_output = xr_holder.copy(data=prd_classes.reshape(xr_holder.shape))
    # Ensuring the same data type as the original raster
    classifier_output = classifier_output.astype(np.float32)

    return classifier_output


# A function to read the classification CSV file provided in the solution to test the model and check the impact of the Polygons mask
# used to extract the classes reflectance
def classify_sentinel_data_sol(
    rasters: xr.DataArray, classified_rgb_rows_sol: Path
) -> ClassificationResult:
    # Loading the classified rgb reflectenace
    df_classified_ref = pd.read_csv(classified_rgb_rows_sol, sep=";")

    # Getting the features reflectance and the labels
    feature_ref = df_classified_ref[["B04", "B03", "B02", "B8A"]].values
    # feature_ref = df_classified_ref[["B04", "B8A"]].values
    feature_label = df_classified_ref.feature_id.values

    # Initiating the classifier, n_job is to speed the process through parallelization
    classifier = RandomForestClassifier(n_jobs=4)  # n_estimators=100, random_state=42)

    # Training the model in the features reflectance and the labels
    classifier.fit(feature_ref, feature_label)

    # Preparing the rasters for prediction by flattening the reflectance data assign it to new dimension variable z and transpose it to match the train data
    flt_data = rasters.stack(z=("x", "y")).T
    # flt_data = np.reshape(rasters.isel(band=[0,3]).stack(z=("x", "y")).T.data, (-1,feature_ref.shape[1]))

    # Predicting the classes of the rasters
    prd_classes = classifier.predict(flt_data)

    # Reshaping the prediction result to match the original shape and put it into [through .copy(data=)] xr.DataArry with the same metadata and attribute as the rasters
    xr_holder = rasters.isel(band=0, drop=True)
    classifier_output = xr_holder.copy(data=prd_classes.reshape(xr_holder.shape).T)
    # Ensuring the same data type as the original raster
    classifier_output = classifier_output.astype(np.float32)

    return classifier_output

In [ ]:
classification_result = classify_sentinel_data(rasters, classified_rgb_rows)

# classify_sentinel_data_sol_path = Path(
#     "D:/Trainings/HW/generated/classified_points_sol_to_read.csv"
# )
# classification_result = classify_sentinel_data_sol(
#     rasters, classify_sentinel_data_sol_path
# )

if verbose:
    log_info(classification_result, "classification_result")
if show_plots:
    plot_array(classification_result, f"{classify_sentinel_data.__name__}")

### (7) Export the classification raster result


In [ ]:
def persist_classification_to_raster(
    raster_output_path: Path, classification_result: ClassificationResult
) -> None:
    # Retrieving the crs_wkt attribute from the spatial_ref dimension (probably unnecessary step as the classification_result is a xr.DataArry with the same metadata and attributes as the raster)
    crs_wkt = classification_result.spatial_ref.crs_wkt
    # Setting the CRS based on the crs_wkt attribute of the spatial_ref dim of the original sentinel raster
    classification_result.rio.write_crs(crs_wkt, inplace=True)
    # Storing the resulted raster of the classification
    classification_result.rio.to_raster(raster_output_path)


In [ ]:
persist_classification_to_raster(raster_output_path, classification_result)
log_info(f"Written Classified Raster to {raster_output_path}")

# --

log_info("Congratulations, you reached the end of the tutorial!")
